In [1]:
import sys
import glob
import os
import pandas as pd
import ee
import numpy as np
import geopandas as gp

import matplotlib.pyplot as plt
import rsfuncs as rs

from scipy import stats
from scipy.signal import savgol_filter
from datetime import datetime, timedelta
from datetime import datetime as dt
from itertools import chain

%load_ext autoreload
%autoreload 2

plt.rcParams['figure.figsize'] = [18,16]
ee.Initialize()

In [2]:
# EE functions are in the rsfuncs module. Loacal functions are here: 

In [3]:
def read_sm_file(filename):
    with open(filename) as f:
        contents = f.readlines()

    data = []
    for line in contents:
        if line[0:1] == "#":
            continue
        else:
            data.append(line)

    headers = [x.replace("Soil Moisture Percent","smp").replace(" ","_") for x in data[0].split(",")]
    cols = [x.strip("\n").split(",") for x in data[1:]]

    df = pd.DataFrame(cols, columns = headers)
    
    return df

def imlist_2_timeseries(imlist,polarization, area):
    
    '''
    This essemtially combines `get_ims_by_date` and `array from latlon` functions below 
    '''
    num_ims = len(imlist.getInfo())

    ims = []
    dates = []
    
    for idx in range (0, num_ims):
        latlon = ee.Image.pixelLonLat().addBands(imlist.get(idx))
        res = latlon.reduceRegion(reducer=ee.Reducer.toList(),geometry=area,maxPixels=1e8,scale=10)
        
        try:
            lats = np.array((ee.Array(res.get("latitude")).getInfo()))
            lons = np.array((ee.Array(res.get("longitude")).getInfo()))
            data = np.array((ee.Array(res.get(polarization)).getInfo()))
        except:
            data = np.full_like(lats, np.nan,dtype=np.float64)

        im = make_np_array(data, lats, lons)
        ims.append(im)
        
        date =  latlon.get('system:time_start')
        info_dict = imlist.get(i).getInfo()
        date = info_dict['id']
        dates.append(date)
        
    return ims, dates

def get_ims_by_date(ims_list, var, res=10):
    imlist = []
    imdates = []
    num_images = len(ims_list.getInfo())

    for i in range (0, num_images):
        
        if i % 5 == 0:
            print(str((i / num_images)*100)[:5] + " % ")

        latlon = ee.Image.pixelLonLat().addBands(ims_list.get(i))
        imlist.append(array_from_latlon(latlon, var, res))
        date =  latlon.get('system:time_start')
        info_dict = ims_list.get(i).getInfo()
        date = info_dict['id']
        imdates.append(date)
  
    return imlist, imdates

def array_from_latlon(latlon_obj, var, res):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.toList(),geometry=area,maxPixels=1e8,scale=res)
    try:
        lats = np.array((ee.Array(res.get("latitude")).getInfo()))
        lons = np.array((ee.Array(res.get("longitude")).getInfo()))
        data = np.array((ee.Array(res.get(var)).getInfo()))
    except:
        data = np.full_like(lats, np.nan,dtype=np.float64)
    
    out = make_np_array(data, lats, lons)
    return out   

def make_np_array(data, lats, lons):
    # get data from df as arrays
    lons = np.array(lons)
    lats = np.array(lats)
    data = np.array(data) # Set var here 
                                              
    # get the unique coordinates
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)

    # get number of columns and rows from coordinates
    ncols = len(uniqueLons)    
    nrows = len(uniqueLats)

    # determine pixelsizes
    ys = uniqueLats[1] - uniqueLats[0] 
    xs = uniqueLons[1] - uniqueLons[0]

    # create an array with dimensions of image
    arr = np.zeros([nrows, ncols], np.float32)

    # fill the array with values
    counter =0
    for y in range(0,len(arr),1):
        for x in range(0,len(arr[0]),1):
            if lats[counter] == uniqueLats[y] and lons[counter] == uniqueLons[x] and counter < len(lats)-1:
                counter+=1
                arr[len(uniqueLats)-1-y,x] = data[counter] # we start from lower left corner
    
    return arr

def filter_date(product,y,m,d):
    start = ee.Date.fromYMD(y,m,d).advance(-1, "day")
    end = ee.Date.fromYMD(y,m,d)
    prod = product.filterDate(start, end).sort('system:time_start', False).select("ppt")
    return prod

def format_dates(dates):
    for idx, x in enumerate(dates):
        timestamp = x.find("V_")+2
        timestr = x[timestamp:timestamp+13]
        dates[idx] = pd.to_datetime(timestr, format='%Y%m%d %H:%M')
    return dates

def get_2day_precip(latlon_obj, area):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.sum(),geometry=area,scale=10)
    data = np.array((ee.Array(res.get("ppt")).getInfo()))
    out = np.array(data)
    return out 

def get_ndvi(latlon_obj, area):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.mean(),geometry=area,scale=10)
    data = np.array((ee.Array(res.get("NDVI")).getInfo()))
    out = np.array(data)
    return out 

def round_up_to_odd(f):
    f = int(np.ceil(f))
    return f + 1 if f % 2 == 0 else f

In [4]:
site_file = gp.read_file("../shape/scan_sites.shp")
sites = site_file[~site_file['state'].isin(["AK", "HI", "PR", "VI"])]

In [5]:
data = rs.load_data()

In [6]:
data_dir = "../data"

In [7]:
out_dict = {}

In [ ]:
# Grab the landsat time series 
   
for idx, row in sites.iterrows():
    
    if row.id in out_dict.keys():
        print(row.id)
        continue
        
    print("Processings site no {}".format(row.id))
    
    # Make geom to submit to EE 
    x,y = row.geometry.buffer(0.0001).envelope.exterior.coords.xy
    coords = [list(zip(x,y))]
    area = ee.Geometry.Polygon(coords)
    
    # Get the corresponding SCAN data file from data folder
    site_id = row.id
    sm_file = [os.path.join(data_dir,x) for x in os.listdir(data_dir) if site_id in x][0]
    sm_dat = read_sm_file(sm_file)
    sm_dat['Date'] =  pd.to_datetime(sm_dat['Date'], format='%Y%m%d %H:%M')
    sm_dat.set_index('Date', inplace=True)
        
    # start and end date
    if sm_dat.empty:
        print("no valid soil moisture data for {}".format(row.id))
        continue
        
    startdate = sm_dat.index[0]
    enddate = sm_dat.index[-1]
    
    date = startdate.strftime("%Y-%m-%d")
        
    # Landsat - Note: some sites are in the overlap areas between passes.
    # these sites can have multiple obs / day or obs separated by 8days instead of 16. 
    
    print("Processing Landsat")
    landsat = rs.load_data()['l8_sr']
    lic, lvar, lsf = landsat[0],landsat[1], landsat[2]

    lstart = ee.Date.fromYMD(startdate.year,startdate.month,startdate.day).advance(-9,"day")
    lend = ee.Date.fromYMD(enddate.year,enddate.month,enddate.day).advance(8, "day")

    l8_col = lic.filterDate(lstart,lend).filterBounds(area).map(rs.mask_quality) # Mask clouds and shadows 
    lt = l8_col.sort('system:time_start')
    lims = lt.toList(lt.size())

    num_ims = len(lims.getInfo())

    ldfs = []

    for i in range(0, num_ims):
        if i % 5 == 0:
            print(str((i / num_ims)*100)[:5] + " % ")

        ls_latlon = ee.Image.pixelLonLat().addBands(lims.get(i))
        ltemp = ls_latlon.select(["B1","B2","B3","B4","B5","B6","B7"]).multiply(lsf)
        l8_res = ltemp.reduceRegion(reducer=ee.Reducer.mean(),geometry=area,bestEffort=True,scale=30)

        l8_info_dict = lims.get(i).getInfo()
        l8_date = l8_info_dict['id'][-8:]

        l8_out = l8_res.getInfo()

        ldf = pd.DataFrame.from_dict(l8_out.values()).T
        ldf.columns = l8_out.keys()
        ldf.index = pd.to_datetime([l8_date])
        ldfs.append(ldf)

    # concat the dfs 
    ls = pd.concat(ldfs)
    print(len(ls.dropna()))

    for i in ls.columns:
        ls[i][ls[i] == 0.] = np.nan
        
    print(len(ls.dropna()))
    
    out_dict[(row.id)] = ls

Processings site no 2057
Processing Landsat
0.0 % 
3.448 % 
6.896 % 
10.34 % 
13.79 % 
17.24 % 
20.68 % 
24.13 % 
27.58 % 
31.03 % 
34.48 % 
37.93 % 
41.37 % 
44.82 % 
48.27 % 
51.72 % 
55.17 % 
58.62 % 
62.06 % 
65.51 % 
68.96 % 
72.41 % 
75.86 % 
79.31 % 
82.75 % 
86.20 % 
89.65 % 
93.10 % 
96.55 % 
145
76
Processings site no 2078
Processing Landsat
0.0 % 
3.676 % 
7.352 % 
11.02 % 
14.70 % 
18.38 % 
22.05 % 
25.73 % 
29.41 % 
33.08 % 
36.76 % 
40.44 % 
44.11 % 
47.79 % 
51.47 % 
55.14 % 
58.82 % 
62.5 % 
66.17 % 
